# Helper Functions for Taxi Time Optimization
### FAA Student Data Challenge

## Matt Brune, Duke University

**Runway End Function**

Returns runway threshold end for operation given operation type and runway

In [4]:
#corrects runway threshold for taxi distance on arrivals
def runway_end(runway, op):
    if ("L" in runway) | ("R" in runway):
        ogLetter = runway[-1]
        runway = runway[:-1]
        if ogLetter == "L":
            letter = "R"
        else:
            letter = "L"
    else:
        letter=""
    if op == "Arrival":
        if int(runway) > 18:
            otherEnd = int(runway) - 18
        else:
            otherEnd = int(runway) + 18
    else:
        otherEnd = int(runway)
    end = str(otherEnd)+letter
    return end

**Data Cleaning Function**

Returns x, y from Aerobahn monthly csv

In [3]:
def taxi_time_clean(month, gateLookup, taxiDist, weatherLookup):
   # this function passes in a pd.DataFrame of a given month-long Aerobahn csv file and airport specific lookups for gating, taxi distances, and weather
   # it returns two dataframes: the predictor variables, and the taxi time target variable

   #filters to only relevant columns (for computation time reduction), and only arr and dep ops
   month[["Operation", "Gate", "Runway", "Actual Off Block Time (Aerobahn)", "Actual Take Off Time (Aerobahn)", "Actual Landing Time (Aerobahn)", "Actual In Block Time (Aerobahn)"]]
   month = month[(month["Operation"]=="Arrival") | (month["Operation"]=="Departure")]

   #convert to timestamps
   dateCols = ["Actual Off Block Time (Aerobahn)", "Actual Take Off Time (Aerobahn)", "Actual Landing Time (Aerobahn)", "Actual In Block Time (Aerobahn)"]
   for i in dateCols:
      month[i] = month[i].apply(pd.to_datetime)

   #lookups
   GateLookupDict = dict(zip(gateLookup['Aerobahn Gate Label'], gateLookup['Terminal']))
   month["Gate Category"] = month["Gate"].map(GateLookupDict)
   month["GR Code"] = month["Gate Category"]+"_"+month["Runway"]
   TotDistDict = dict(zip(taxiDist['GR Code'], taxiDist['Tot Dist']))
   AngleSumDict = dict(zip(taxiDist['GR Code'], taxiDist['Angle Sum']))
   month["Distance"] = month["GR Code"].map(TotDistDict)
   month["Angle Sum"] = month["GR Code"].map(AngleSumDict)

   month["Hour"] = ""
   month["MDH Code"] = ""
   for i in range(len(month["Operation"])):
      if month["Operation"].iloc[i] == "Departure":
         timeRef = month["Actual Off Block Time (Aerobahn)"].iloc[i]
         month["Hour"].iloc[i] = str(timeRef.hour)
         month["MDH Code"].iloc[i] = str(timeRef.month)+"-"+str(timeRef.day)+"-"+str(timeRef.hour)
      else:
         timeRef = month["Actual Landing Time (Aerobahn)"].iloc[i]
         month["Hour"].iloc[i] = str(timeRef.hour)
         month["MDH Code"].iloc[i] = str(timeRef.month)+"-"+str(timeRef.day)+"-"+str(timeRef.hour)
   month = month.loc[month["Hour"] != 'nan']

   TempDict = dict(zip(weatherLookup['MDH'], weatherLookup['Temp']))
   WindSpeedDict = dict(zip(weatherLookup['MDH'], weatherLookup['WindSpeed']))
   VisibilityDict = dict(zip(weatherLookup['MDH'], weatherLookup['Visibility']))
   month["Temperature"] = month["MDH Code"].map(TempDict)
   month["Wind Speed"] = month["MDH Code"].map(WindSpeedDict)
   month["Visibility"] = month["MDH Code"].map(VisibilityDict)


   #filter to only include  valid gate assignments
   month = month[month["Distance"].notna()]

   

   ## instantiating rest of predictor variables

   #indicator predictor variable
   month["Departure_Indicator"] = [1 if i=="Departure" else 0 for i in month["Operation"]]
   

   #ARR predictor variables & target
   month["nArrOut"] = [len(month[(month["Operation"]=="Arrival") & (month["Actual Off Block Time (Aerobahn)"]<i) & (month["Actual Take Off Time (Aerobahn)"]>i)]) for i in month["Actual Landing Time (Aerobahn)"]]
   month["nArrIn"] = [len(month[(month["Operation"]=="Arrival") & (month["Actual Landing Time (Aerobahn)"]<i) & (month["Actual In Block Time (Aerobahn)"]>i)]) for i in month["Actual Landing Time (Aerobahn)"]]
   month["Taxi In"] = month["Actual In Block Time (Aerobahn)"] - month["Actual Landing Time (Aerobahn)"]

   #DEP predictor variables & target
   month["nDepOut"] = [len(month[(month["Operation"]=="Departure") & (month["Actual Off Block Time (Aerobahn)"]<i) & (month["Actual Take Off Time (Aerobahn)"]>i)]) for i in month["Actual Off Block Time (Aerobahn)"]]
   month["nDepIn"] = [len(month[(month["Operation"]=="Departure") & (month["Actual Landing Time (Aerobahn)"]<i) & (month["Actual In Block Time (Aerobahn)"]>i)]) for i in month["Actual Off Block Time (Aerobahn)"]]
   month["Taxi Out"] = month["Actual Take Off Time (Aerobahn)"] - month["Actual Off Block Time (Aerobahn)"]

   #total taxi time
   month["Taxi Time"] = ""
   for i in range(len(month["Operation"])):
      month["Runway"].iloc[i] = runway_end(month["Runway"].iloc[i], month["Operation"].iloc[i])
      if month["Operation"].iloc[i]=="Departure":
         month["Taxi Time"].iloc[i] = month["Taxi Out"].iloc[i]
      else:
         month["Taxi Time"].iloc[i] = month["Taxi In"].iloc[i]
   month["Taxi Time"] = month["Taxi Time"]/pd.to_timedelta(1, unit="min")

   #REDUCE to just indicator variables & targets
   x = month[["Departure_Indicator", "Distance", "Angle Sum", "nArrOut", "nArrIn", "nDepOut", "nDepIn", "Temperature", "Wind Speed", "Visibility"]]
   y = month["Taxi Time"]

   return x, y

**Scaling Function**

Given a dataframe of x samples, returns a dataframe with min-max scaling

In [7]:
#normalizing data
def data_scaler(x):
    mmScaler = preprocessing.MinMaxScaler()
    mmScaler.fit(x)
    x = mmScaler.transform(x)
    x = pd.DataFrame(x, columns=["Departure_Indicator", "Distance", "Angle Sum", "nArrOut", "nArrIn", "nDepOut", "nDepIn", "Temperature", "Wind Speed", "Visibility"])
    return x

**Metric Computation Function**

Given a model, x and y samples, and a cross-validation, outputs $R^{2}$ and RMSE

In [1]:
def metric_disp(model, x, y, crossval):
    R2 = np.mean(cross_val_score(model, x, y, scoring=None, cv=crossval))
    RMSE = np.mean(np.sqrt(np.abs(cross_val_score(model, x, y, scoring = 'neg_mean_squared_error', cv = crossval))))
    return R2, RMSE

**Model Selection Results Function**

Given a list of models, x and y samples, and a cross validation, outputs $R^{2}$ and RMSE for each model

In [9]:
def model_selection_results(models, x, y, crossval):
    file1 = open("IAHresults.txt","w")
    for i in models:
        r2, rmse = metric_disp(i, x, y, crossval)
        file1.write("For the {} model: \n R^2: {} \n RMSE: {} \n".format(i, r2, rmse))
    file1.close()

In [56]:
def model2Q_selection_results(models, x, y, crossval):
    file1 = open("IAH2Qresults.txt","w")
    for i in models:
        r2, rmse = metric_disp(i, x, y, crossval)
        file1.write("For the {} model: \n R^2: {} \n RMSE: {} \n".format(i, r2, rmse))
    file1.close()

**June 2022 Testing Results Function**

Given the list of tuned models, x and y samples for 

In [7]:
def june22_results(models, xTrain, yTrain, xTestEWR, yTestEWR, xTestIAH, yTestIAH):
    file2 = open("June2022Results.txt", "w")
    for i in models:
        file2.write("Newark {}: \n".format(i))
        file2.write("R^2: {} \n".format(i.fit(xTrain, yTrain).score(xTestEWR, yTestEWR)))
        file2.write("RMSE: {} \n".format(np.sqrt(mean_squared_error(i.predict(xTestEWR),yTestEWR))))
        file2.write("Houston {} \n".format(i))
        file2.write("R^2: {} \n".format(i.fit(xTrain, yTrain).score(xTestIAH, yTestIAH)))
        file2.write("RMSE: {} \n".format(np.sqrt(mean_squared_error(i.predict(xTestIAH),yTestIAH))))
    file2.close()

In [52]:
def june22_2Qresults(models, xTrain, yTrain, xTestEWR, yTestEWR, xTestIAH, yTestIAH):
    file2 = open("June2022_2QResults.txt", "w")
    for i in models:
        file2.write("Newark {}: \n".format(i))
        file2.write("R^2: {} \n".format(i.fit(xTrain, yTrain).score(xTestEWR, yTestEWR)))
        file2.write("RMSE: {} \n".format(np.sqrt(mean_squared_error(i.predict(xTestEWR),yTestEWR))))
        file2.write("Houston {} \n".format(i))
        file2.write("R^2: {} \n".format(i.fit(xTrain, yTrain).score(xTestIAH, yTestIAH)))
        file2.write("RMSE: {} \n".format(np.sqrt(mean_squared_error(i.predict(xTestIAH),yTestIAH))))
    file2.close()

**Departure Runway Processing Time Lookup Function**

Given the airport and meteorological conditions, returns runway processing time for *entire* runway system

In [53]:
def dep_rpt_lookup(airport, mc, priority=None):
    if airport == "IAH":
        if mc == "Visual":
            return 60*2/64
        elif mc == "Marginal":
            return 60*2/64
        elif mc == "Instrument":
            return 60*2/64
        else:
            return "Invalid input: ensure airport code is in IATA format, meteorological conditions are one of [Visual, Marginal, Instrument], and priority is one of [Departures, Arrivals]"
    elif airport == "EWR":
        if (mc == "Visual") & (priority=="Departures"):
            return 60*2/52
        elif mc == "Visual" & priority=="Arrivals":
            return 60/42
        elif mc == "Marginal":
            return 60/42
        elif mc == "Instrument":
            return 60/34
        else:
            return "Invalid input: ensure airport code is in IATA format, meteorological conditions are one of [Visual, Marginal, Instrument], and priority is one of [Departures, Arrivals]"
    else:
        return "Invalid input: ensure airport code is in IATA format, meteorological conditions are one of [Visual, Marginal, Instrument], and priority is one of [Departures, Arrivals]"


**Runway Processing Time Transform**

Given an airport, meteorological conditions, an optional focus for EWR, and the x and y-values, returns the "2Q" taxi times from the total taxi-out times.

In [77]:
def rpt_transform(airportcode, conditions, x, y, focus=None):
    #find runway processing times
    rpt = dep_rpt_lookup(airportcode, conditions, focus)
    qRed = 1
    y2 = y.copy()

    for i in x.index:
    #determine queue length in front of aircraft pushing back
        qt = max(0, rpt*qRed*x.loc[i, "nDepOut"] - (x.loc[i, "Distance"] / 1519))
    #subtract out from taxi times
        y2.loc[i] = max(1, y2.loc[i]-qt)
    
    rptLog = np.log(y2)
    
    return rptLog

**Cost Function Plotting**

Given the cost type, slope, a scale factor, the x-values, and the optimal pushback policy, returns y-values for plotting on newsvendor plot

In [78]:
def cost_plot(type, slope, scaler, xvals, optimal):
    yvals = np.zeros(len(xvals))
    if type=="A":
        for i in range(len(xvals)):
            if xvals[i] <= optimal:
                yvals[i] = (slope/scaler) * (abs(xvals[i]-optimal))
            else:
                yvals[i] = -1
    elif type=="B":
        for i in range(len(xvals)):
            if xvals[i] <= optimal:
                yvals[i] = -1
            else:
                yvals[i] = (slope/scaler) * (xvals[i]-optimal)
    return yvals

**Sampling 2Q Function**

Given a row value, x and y values, a fitted model, the method of error, and the cost ratio of runway underutilization to the queueing cost, plots the optimal pushback interval, and returns a string listing the optimal value.

In [84]:
def twoQ_sampler(rowval, x, model, error, costratio):
    if type(rowval) != int:
        return "Row Value error: enter an integer"
    else:
        sample = x.iloc[rowval,:]
        sampleMu = np.exp(model.predict(np.array(sample).reshape(1,-1)))
        sampleSigma = error

        costA = costratio
        costB = 1
        cumProb = (costB/(costA+costB))
        T = lognorm.ppf(q=cumProb, scale=sampleMu, s=sampleSigma)
        plotX = np.arange(0,30,0.01)

        fig, ax = plt.subplots(1, 1, clear=True, figsize=(10,6))
        ax.plot(plotX, lognorm.pdf(x=plotX, scale=sampleMu, s=sampleSigma), label="Lognormal Distribution of 2Q Taxi Times")
        ax.plot(plotX, cost_plot("A", costA, 50, plotX, T), label="Runway Underutilization Cost")
        ax.plot(plotX, cost_plot("B", costB, 50, plotX, T), label="Queueing Cost")
        ax.axvline(x=T, color='k', ymin=0, ls='--', label="Optimal Pushback Interval")
        ax.text(15,(max(lognorm.pdf(x=plotX, scale=sampleMu, s=sampleSigma))+0.05)/4, 'With the given optimal pushback interval, \nthe optimal pushback rate is: \n{} aircraft per 15 minutes'.format(round(15/T[0], 0)), style ='italic', fontsize = 12, horizontalalignment = "center", bbox ={'facecolor':'green','alpha':0.5, 'pad':10})
        ax.set_title("'2Q' Distribution for Sample #{} from June EWR Test Data, with Cost Ratio = {}:{}".format(rowval, costA,costB))
        ax.legend(loc="upper right")
        ax.set_xlim(0,20)
        ax.set_ylim(0,max(lognorm.pdf(x=plotX, scale=sampleMu, s=sampleSigma))+0.05)
        ax.set_xlabel("Taxi Times (min)")
        fig.savefig("samplePlot.jpg")
        plt.show()

    return "Optimal pushback interval is {} minutes.".format(round(T[0], 3))